In [7]:
# 모듈 import
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

In [8]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
#cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
cfg=Config.fromfile('./configs/yolof/yolof_r50_c5_8x8_1x_coco.py')
#cfg=Config.fromfile('./configs/yolof/yolof_r50_c5_8x8_iter-1x_coco.py')
root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
#stratified kfold
cfg.data.train.ann_file=root+'stratified_v2/train_fold_1.json'
#cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize



cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
#stratified k fold
cfg.data.val.classes = classes
cfg.data.val.img_prefix=root
cfg.data.val.ann_file=root+'stratified_v2/val_fold_1.json'
print(cfg.data.val.pipeline)
cfg.data.val.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
#cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'
cfg.work_dir='./work_dirs/yolof-iter-noiter'

#cfg.model.roi_heads.bbox_head.num_classes = 10
cfg.model.bbox_head.num_classes = 10
#직접수정함
#cfg.optimizer.lr=0.00001

#파라미터 수정함 (학습 너무 많아서 바꿈)
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

[{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (512, 512), 'flip': False, 'transforms': [{'type': 'Resize', 'keep_ratio': True}, {'type': 'RandomFlip'}, {'type': 'Normalize', 'mean': [103.53, 116.28, 123.675], 'std': [1.0, 1.0, 1.0], 'to_rgb': False}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img']}]}]


In [9]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]
#valid까지 확인
#datasets=[build_dataset(cfg.data.train),build_dataset(cfg.data.val)]

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


In [10]:
# dataset 확인
datasets[0]



CocoDataset Train dataset with number of images 3914, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3161  | 1 [Paper]     | 5115  | 2 [Paper pack]  | 706   | 3 [Metal]   | 769   | 4 [Glass]    | 835   |
| 5 [Plastic]       | 2350  | 6 [Styrofoam] | 1026  | 7 [Plastic bag] | 4151  | 8 [Battery] | 143   | 9 [Clothing] | 377   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [11]:
# 모델 build 및 pretrained network 불러오기

model = build_detector(cfg.model)
#model.init_weights()

In [12]:
# #wandb테스트 여기서 이렇게 코드를 쳐서 실행하나 ./configs/_base_/default_runtime.py에서 수정하나 똑같은듯?
# import wandb
# wandb.init(project="level2-objectdetection", entity="hi-ai")
# cfg.log_config.hooks=[
#     dict(type='TextLoggerHook'),
#     dict(type='MMDetWandbHook',
#          init_kwargs={'project':'level2-objectdetection'},
#          interval=10,
#          log_checkpoint=True,
#          log_checkpoint_metadata=True,
#          num_eval_images=100)]

import wandb
wandb.init(project="Model", entity="hi-ai")

log_config = dict(
    interval=10,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(
            type='WandbLoggerHook',
            init_kwargs=dict(project='Model', name='yolof', entity='hi-ai')
            )])
# for wandb
cfg.log_config = log_config

# import wandb
# wandb.init(project="level2-objectdetection", entity="hi-ai")


# 모델 학습

train_detector(model, datasets, cfg, distributed=False, validate=True)

# preds = []
# gts = []

# # 테스트 데이터의 예측 결과를 추출합니다.
# for i, data in enumerate(data_loader):
#     with torch.no_grad():
#         result = single_gpu_test(model, data["img"][0].cuda(), show=False)
#     preds.extend(result[0])
#     gts.extend(data["gt_labels"][0].tolist())

# # confusion matrix와 accuracy를 계산합니다.
# cm = eval_map(preds, gts, iou_thr=0.5, print_summary=False, dataset=dataset.CLASSES)
# acc = sum([1 for i, j in zip(preds, gts) if i == j]) / len(preds)

# wandb에 confusion matrix와 accuracy를 로깅합니다.
# wandb.log({"confusion_matrix": wandb.plot.confusion_matrix(preds, gts, class_names=dataset.CLASSES),
#            "accuracy": acc})

epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅██
learning_rate,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇█████████████
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/grad_norm,█▇▅▄▃▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▇▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_bbox,▃▅▅█▅▅▆▆▃▅▂▆▆▆▆▆▃▅▄▃▁▄▅▄▃▃▃▇▃▃▄▁▅▆▃▄▃▂▃▁
train/loss_cls,█▇▅▄▃▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/bbox_mAP,▁▁
val/bbox_mAP_50,▁█
val/bbox_mAP_75,▁▁
val/bbox_mAP_l,▁▁


2023-05-11 07:39:34,476 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2023-05-11 07:39:34,593 - mmdet - INFO - Start running, host: root@718ec73291f9, work_dir: /opt/ml/baseline/mmdetection/work_dirs/yolof-iter-noiter
2023-05-11 07:39:34,616 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


2023-05-11 07:39:51,307 - mmdet - INFO - Epoch [1][10/979]	lr: 7.995e-04, eta: 1:34:02, time: 0.481, data_time: 0.291, memory: 1952, loss_cls: 5.2862, loss_bbox: 0.7241, loss: 6.0103, grad_norm: 119.5148
2023-05-11 07:39:53,324 - mmdet - INFO - Epoch [1][20/979]	lr: 1.599e-03, eta: 1:06:42, time: 0.202, data_time: 0.015, memory: 1952, loss_cls: 2.1949, loss_bbox: 0.8265, loss: 3.0214, grad_norm: 32.1329
2023-05-11 07:39:55,511 - mmdet - INFO - Epoch [1][30/979]	lr: 2.398e-03, eta: 0:58:40, time: 0.219, data_time: 0.017, memory: 1952, loss_cls: 1.9541, loss_bbox: 0.7935, loss: 2.7476, grad_norm: 23.4352
2023-05-11 07:39:57,493 - mmdet - INFO - Epoch [1][40/979]	lr: 3.198e-03, eta: 0:53:37, time: 0.198, data_time: 0.018, memory: 1952, loss_cls: 1.1202, loss_bbox: 0.9522, loss: 2.0723, grad_norm: 16.0595
2023-05-11 07:39:59,508 - mmdet - INFO - Epoch [1][50/979]	lr: 3.997e-03, eta: 0:50:43, time: 0.202, data_time: 0.013, memory: 1952, loss_cls: 1.0913, loss_bbox: 0.7810, loss: 1.8723, gra

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 969/969, 30.7 task/s, elapsed: 32s, ETA:     0s

...

2023-05-11 07:52:20,552 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.004
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.016
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.005
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.070
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.070
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.070
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=2.87s).


2023-05-11 07:52:25,558 - mmdet - INFO - Epoch [4][10/979]	lr: 1.200e-01, eta: 0:30:36, time: 0.484, data_time: 0.303, memory: 1952, loss_cls: 0.5915, loss_bbox: 0.7094, loss: 1.3009, grad_norm: 0.5874
2023-05-11 07:52:27,443 - mmdet - INFO - Epoch [4][20/979]	lr: 1.200e-01, eta: 0:30:34, time: 0.189, data_time: 0.013, memory: 1952, loss_cls: 0.5590, loss_bbox: 0.6972, loss: 1.2561, grad_norm: 0.4045
2023-05-11 07:52:29,489 - mmdet - INFO - Epoch [4][30/979]	lr: 1.200e-01, eta: 0:30:31, time: 0.205, data_time: 0.016, memory: 1952, loss_cls: 0.5674, loss_bbox: 0.7573, loss: 1.3247, grad_norm: 0.4483
2023-05-11 07:52:31,320 - mmdet - INFO - Epoch [4][40/979]	lr: 1.200e-01, eta: 0:30:29, time: 0.183, data_time: 0.010, memory: 1952, loss_cls: 0.5453, loss_bbox: 0.7191, loss: 1.2643, grad_norm: 0.4152
2023-05-11 07:52:33,213 - mmdet - INFO - Epoch [4][50/979]	lr: 1.200e-01, eta: 0:30:26, time: 0.189, data_time: 0.008, memory: 1952, loss_cls: 0.5712, loss_bbox: 0.7055, loss: 1.2767, grad_nor

KeyboardInterrupt: 

--- Logging error ---
Traceback (most recent call last):
  File "/opt/conda/envs/detection/lib/python3.7/logging/__init__.py", line 1029, in emit
    self.flush()
  File "/opt/conda/envs/detection/lib/python3.7/logging/__init__.py", line 1009, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "/opt/conda/envs/detection/lib/python3.7/threading.py", line 890, in _bootstrap
    self._bootstrap_inner()
  File "/opt/conda/envs/detection/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/detection/lib/python3.7/site-packages/wandb/sdk/service/streams.py", line 49, in run
    self._target(**self._kwargs)
  File "/opt/conda/envs/detection/lib/python3.7/site-packages/wandb/sdk/internal/internal.py", line 174, in wandb_internal
    logger.error(f"Thread {thread.name}:", exc_info=exc_info)
Message: 'Thread WriterThread:'
Arguments: ()
Thread WriterThread:
Traceback (most recent call last):
  File "/opt/co